In [ ]:
import pandas as pd
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [ ]:
#Used to handle API keys
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=r"C:\Users\danie\OneDrive\Documentos\GitHub\book_recommender\.env")
print("OPENAI_API_KEY loaded:", os.getenv("OPENAI_API_KEY") is not None)

In [ ]:
import pandas as pd

books = pd.read_csv('books_cleaned.csv')

books

In [ ]:
#langchain functions do not work with pandas so we need to create a text file with the tagged descriptions
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   header = False,
                                   index = False)

In [ ]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()

#Text file will be splitted to have one record per chunk
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [ ]:
documents[0]

In [ ]:
embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))

db_books = Chroma.from_documents(
    documents,
    embedding=embeddings)



In [ ]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

In [ ]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

In [ ]:
#Function that retrieves the information about the books that match the query prompt

def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)

In [ ]:
retrieve_semantic_recommendations("A book to teach children about nature")